In [97]:
from typing_extensions import Self
from cv2 import SimpleBlobDetector_Params
import pandas as pd
from pip import main
from scipy.spatial.distance import cosine
import numpy as np
from numpy.linalg import norm
from sklearn.metrics.pairwise import linear_kernel

from TF_IDF.TF_IDF import TF_IDF
from sklearn.metrics.pairwise import cosine_similarity

class RS:

    def __init__(self) -> None:
        self.data = pd.read_csv('C:\\Users\\quach\\Desktop\\Personal\\FPT University\\SEMESTER 9\\Dataset\\student_data.csv')
        self.all_user = self.data.iloc[:,:1].to_numpy().flatten()
        self.SIM_matrix = pd.DataFrame(index=self.all_user,columns=self.all_user)
        self.TF_IDF = TF_IDF()

    def corr_cosine(self, util_matrix):
        return cosine_similarity(util_matrix,util_matrix)
        # SIM_matrix = pd.DataFrame(index=self.all_user,columns=self.all_user)
        # for i in range(0,util_matrix.__len__()):
        #     for j in range(0,util_matrix.__len__()):
        #         SIM_matrix.iloc[i][j] = cosine_similarity(  [util_matrix.iloc[i,1:].to_numpy()],
        #                                                     [util_matrix.iloc[j,1:].to_numpy()])
        # return SIM_matrix

    def corr_euclidean(self, util_matrix):
        SIM_matrix = pd.DataFrame(index=self.all_user,columns=self.all_user)
        for i in range(0,util_matrix.__len__()):
            for j in range(0,util_matrix.__len__()):
                SIM_matrix.iloc[i][j] = np.linalg.norm(util_matrix.iloc[i,1:].to_numpy() - util_matrix.iloc[j,1:].to_numpy())
        return SIM_matrix

    def cosine_sim(self,vec):
        return linear_kernel(vec, vec)

    def compute_all_corr(self):
        SIM_cp = self.data[["Cleanliess","Privacy"]].to_numpy()
        print(self.corr_cosine(SIM_cp))

        #print(SIM_cp.dot(SIM_cp.T)[0]) # simcp * simcp.T
        # cosine_similarity_pd = pd.DataFrame(cosine_similarity, columns = [*range(len(a))])
        # print(cosine_similarity_pd)
        #print(self.corr_cosine(SIM_cp)[0])
        # vec_food_drink = self.TF_IDF.text2vec(self.data["food_drink"])
        # return vec_food_drink
        #sim_food_drink =  self.corr_euclidean(vec_food_drink)
        # return self.cosine_sim(vec,vec)
        #corr_matrix()

if __name__ == "__main__":
    # RS = RS()
    # RS.compute_all_corr()

    # res = RS.compute_all_corr()
    # print(res[0])
    # print("FINISH...")
    # from sklearn.metrics.pairwise import linear_kernel
    # a = [[1,2,3]]
    # b = [[2,4,5]]
    # cosine_similarity = linear_kernel(a, b)
    # print(cosine_similarity)

    # a = [[1,2,3],[2,9,3]]
    # cosine_similarity = linear_kernel(a, a)
    # print(cosine_similarity)
    # cosine_similarity_pd = pd.DataFrame(cosine_similarity, columns = [*range(len(a))])
    # print(cosine_similarity_pd)
    
    cos = cosine_similarity([[12,41,60,11,21],[40,11,4,11,14]],[[12,41,60,11,21],[40,11,4,11,14]])
    print(cos)


    

[[1.         0.45227058]
 [0.45227058 1.        ]]
